In [8]:
pip install ollama


Note: you may need to restart the kernel to use updated packages.


In [10]:
import ollama
import json

def analyze_feedback(feedback_list):
    # Step 1: Convert the list to a single string
    feedback_text = "\n".join(f"- {fb}" for fb in feedback_list if fb.strip())

    # Step 2: Prompt to summarize key issues
    prompt_summary = f"""Here is a list of student complaints:\n{feedback_text}\n\nSummarize these feedbacks into key issues."""
    
    summary_response = ollama.chat(
        model='llama3',
        messages=[{"role": "user", "content": prompt_summary}]
    )
    summary = summary_response['message']['content'].strip()

    # Step 3: Prompt to generate improvement suggestions in JSON
    prompt_improve = f"""
Here are the summarized student feedback issues:

{summary}

Return a JSON array where each object contains:
- "issue": the specific issue
- "suggestion": one actionable suggestion for that issue

Respond ONLY with valid JSON and no additional explanation.
"""
    
    suggestion_response = ollama.chat(
        model='llama3',
        messages=[{"role": "user", "content": prompt_improve}]
    )
    
    suggestions_raw = suggestion_response['message']['content'].strip()

    # Step 4: Try to parse JSON safely
    try:
        suggestions_json = json.loads(suggestions_raw)
    except json.JSONDecodeError as e:
        print("⚠️ JSON parsing failed. Raw output:")
        print(suggestions_raw)
        suggestions_json = []

    return summary, suggestions_json

# Example usage:
negative_feedbacks = [
    "The instructor don't use any examples for explanation.",
    "Explanation is poor.",
    "The teaching is slow. They stretch the same thing for a long time, and students lose interest.",
    "Not good",
    "Cooperation needed with students",
    "I can't understand the method of teaching",
    ""
]

summary, suggestions = analyze_feedback(negative_feedbacks)

print("📌 Summary of Issues:\n", summary)
print("\n💡 Suggested Improvements (JSON):\n")
for item in suggestions:
    print(f"Issue: {item['issue']}\nSuggestion: {item['suggestion']}\n")


📌 Summary of Issues:
 Based on the student complaints, the key issues that emerge are:

1. **Lack of effective explanation**: Students feel that the instructor's explanations are poor and don't provide sufficient examples to help them understand the material.
2. **Slow pace of teaching**: The instructor is taking too long to cover a single topic, leading to students losing interest and attention.
3. **Difficulty understanding teaching method**: Some students are struggling to grasp the instructor's teaching approach or methodology.

These key issues suggest that students are having trouble engaging with the course material and finding it difficult to learn effectively due to the instructor's teaching style and pace.

💡 Suggested Improvements (JSON):

Issue: Lack of effective explanation
Suggestion: Provide more concrete examples or visual aids to help students understand complex concepts

Issue: Slow pace of teaching
Suggestion: Break topics into smaller, manageable chunks and allow ti